In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
from datetime import datetime

# Quick Books Df 
# Upload Quickbooks data - Copy path from file and paste within Quotations
# If Quickbooks Data and COGs Data are always named the same, only would need to upload files to Google Collab and Run All
df = pd.read_excel('/content/Quickbooks Data.xlsx')
# COGs DF
# Upload COGs For All Products 
# To get list of all Unique Products - uncomment / download prodtypeDf
otherDf = pd.read_excel('/content/COGS_Prices for Products.xlsx')
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Type,Unnamed: 7,Date,Unnamed: 9,Num,Unnamed: 11,Memo,Unnamed: 13,Name,Unnamed: 15,Qty,Unnamed: 17,U/M,Unnamed: 19,Sales Price,Unnamed: 21,Amount,Unnamed: 23,Balance
0,NaN,Parts,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Cali Coast - Flower - 7.0g-E,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,Invoice,NaN,2021-01-12,NaN,E1443254,NaN,Cali Coast - Flower - Cali Coast Flo - 7.0g,NaN,Cali Coast South ( MNR),NaN,64.0,NaN,ea,NaN,25.0,NaN,1600.0,NaN,1600.0
3,NaN,NaN,NaN,NaN,NaN,NaN,Invoice,NaN,2021-01-12,NaN,E1443254,NaN,Cali Coast - Flower - Strawberry Pablo - 7.0g,NaN,Cali Coast South ( MNR),NaN,128.0,NaN,ea,NaN,25.0,NaN,3200.0,NaN,4800.0
4,NaN,NaN,NaN,NaN,NaN,NaN,Invoice,NaN,2021-03-30,NaN,E1742510,NaN,Cali Coast - Flower Smalls - Cali Coast Margy ...,NaN,The Medical Strain Ventures,NaN,128.0,NaN,ea,NaN,25.0,NaN,3200.0,NaN,8000.0


In [ ]:
%pip install xlsxwriter
from pandas import ExcelWriter
import xlsxwriter

In [ ]:
# Create COGs DF
COGsDf = otherDf.dropna()
COGsDf = COGsDf.rename(columns={'Unnamed: 1' :'COGs'})
COGsDf

,Memo,COGs,Type
0,Cali Coast - Pre-Roll - Hybrid - 1.0g,2.50,Cali Coast - Pre-Roll 1pk
1,Cali Coast - Caesar OG - 3.5g jars,7.92,Cali Coast - Flower Jar (3.5g)
2,Cali Coast - Cali Coast - 3.5g jars,7.92,Cali Coast - Flower Jar (3.5g)
3,Cali Coast - Cali Coast - Cali Coast Gold OG -...,7.92,Cali Coast - Flower Jar (3.5g)
4,Cali Coast - Cali Coast Cake - 3.5g jars,7.92,Cali Coast - Flower Jar (3.5g)
...,...,...,...
672,Tyler's Treats - Pre-Roll 5pk - Hybrid - 5 x 1.0g,6.00,Tyler's Treats - Pre-roll 5pk
673,Tyler's Treats - Pre-Roll 5pk - Indica - 5 x 1.0g,6.00,Tyler's Treats - Pre-roll 5pk
674,Tyler's Treats - Shake - Hybrid - 1oz,15.00,Tyler's Treats - Shake (1oz)
675,Tyler's Treats - Shake - Indica - 1oz,15.00,Tyler's Treats - Shake (1oz)


In [ ]:
#Drop unneeded columns
df1 = df.drop(['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 7','Unnamed: 9','Unnamed: 11','Unnamed: 13','Unnamed: 15','Unnamed: 17','Unnamed: 19','Unnamed: 21','Unnamed: 23','U/M','Balance'], axis=1)
# Drop Null values
df2 = df1.dropna()
# DF3 only containes what we want 
df3 = df2.loc[df2.Memo.str.contains("Cali Coast|Dab Daddy|Dab daddy|Everygreen Organix|Evolve|Harvist|Kaviar|Octavius|Peaches|Route 66|Smyle|Super Bad|Superbad|Trees|Tyler's Treats|Velox"),:]
#--Using position to slice Email using a lambda function
df3['pos'] = df3['Memo'].str.find('-')
df3['Company'] = df3.apply(lambda x: x['Memo'][0:x['pos']],axis=1)
df3['Product'] = df3.apply(lambda x: x['Memo'][x['pos']+1:],axis=1)
df3['pos P'] = df3['Product'].str.find('-')
df3['Product type'] = df3.apply(lambda x: x['Product'][0:x['pos P']],axis=1)
# Tell if sample or not
df3['sample'] = np.where(df3['Sales Price'] == .01, 1,0)
# What type of sample
df3['Small Sample'] = np.where(df3['Amount'] < .50, 1,0)
df3['BOGO'] = np.where((df3['Amount'].between(.5,10)), 1,0)
# Removing Cali Coast South, Making you smile, Mather Natures Remedy Caregivers
df4 = df3[df3['Name'] != 'Cali Coast South ( MNR)']
df5 = df4[df4['Name'] != 'Making You Smile']
df6 = df5[df5['Name'] != 'Mother Natures Remedy Caregivers']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [ ]:
# Merging COGs and Df3 
dfMerge = df6.merge(COGsDf, on='Memo', how='left').fillna(0)

# Total COGs
dfMerge['Total COGs'] = dfMerge['COGs'] * dfMerge['Qty']
# Samples in Order
dfMerge['Samples in Order'] = dfMerge['Qty'] * dfMerge['sample']


# Needs to be less than 32 characters long for Excel Sheet names
def char_len(x, fixed_n):
    '''set string x to fixed_n character, prepend with 'xxx' if short'''
    if len(x) > fixed_n: 
        return x[:fixed_n]  
    return x 
# setting max characteres of names 
dfMerge['Name'] = dfMerge['Name'].apply(lambda x: char_len(x,31))
# COGS for samples
dfMerge['Small Sample COGs'] = dfMerge['Small Sample'] * dfMerge['Total COGs']
# Buy One Get One COGs
dfMerge['BOGO COGs'] = dfMerge['BOGO'] * dfMerge['Total COGs']
# Replacing Sales Price for samples to 0
dfMerge['Sales Price'] = dfMerge['Sales Price'].replace(.01,0)
# Real Amount = Real Revenue 
dfMerge['Real Amount'] = dfMerge['Sales Price'] * dfMerge['Qty']
# Profit = Revenue minus Total COGs Internal 
dfMerge['Profit'] = dfMerge['Real Amount'] - dfMerge['Total COGs']
#dfMerge['sample commission'] = dfMerge['sample']*dfMerge['Amount']*.05

dfMerge['Sales Commission'] = (dfMerge['Real Amount']*.05) 

dfMerge

,Type_x,Date,Num,Memo,Name,Qty,Sales Price,Amount,pos,Company,Product,pos P,Product type,sample,Small Sample,BOGO,COGs,Type_y,Total COGs,Samples in Order,Small Sample COGs,BOGO COGs,Real Amount,Profit,Sales Commission
0,Invoice,2021-03-30,E1742510,Cali Coast - Flower Smalls - Cali Coast Margy ...,The Medical Strain Ventures,128.0,25.0,3200.00,11,Cali Coast,Flower Smalls - Cali Coast Margy - 7.0g,15,Flower Smalls,0,0,0,12.47,Cali Coast - Flower Smalls (7g),1596.16,0.0,0.00,0.0,3200.0,1603.84,160.0
1,Invoice,2021-01-06,E1416626,Dab Daddy - BHO - Badder - Gasoline Alley - 1.0g,"Solister, INC",2.0,0.0,0.02,10,Dab Daddy,BHO - Badder - Gasoline Alley - 1.0g,5,BHO,1,1,0,6.97,Dab Daddy - Badder,13.94,2.0,13.94,0.0,0.0,-13.94,0.0
2,Invoice,2021-01-07,E1429368,Dab Daddy - BHO - Badder - Gasoline Alley - 1.0g,Red Moon,20.0,10.0,200.00,10,Dab Daddy,BHO - Badder - Gasoline Alley - 1.0g,5,BHO,0,0,0,6.97,Dab Daddy - Badder,139.40,0.0,0.00,0.0,200.0,60.60,10.0
3,Invoice,2021-01-12,E1434707,Dab Daddy - BHO - Badder - Gasoline Alley - 1.0g,San Fernando Valley Patients Co,10.0,0.0,0.10,10,Dab Daddy,BHO - Badder - Gasoline Alley - 1.0g,5,BHO,1,1,0,6.97,Dab Daddy - Badder,69.70,10.0,69.70,0.0,0.0,-69.70,0.0
4,Invoice,2021-01-18,E1441626,Dab Daddy - BHO - Badder - Gasoline Alley - 1.0g,Norcanna Inc.,1.0,0.0,0.01,10,Dab Daddy,BHO - Badder - Gasoline Alley - 1.0g,5,BHO,1,1,0,6.97,Dab Daddy - Badder,6.97,1.0,6.97,0.0,0.0,-6.97,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834,Invoice,2021-05-28,E1996241,Velox - NanoSerum - 7.0mL,Reefer Madness,1.0,0.0,0.01,6,Velox,NanoSerum - 7.0mL,11,NanoSerum,1,1,0,7.12,Velox - NanoSerum,7.12,1.0,7.12,0.0,0.0,-7.12,0.0
1835,Invoice,2021-06-15,E2061510,Velox - NanoSerum - 7.0mL,"LAHC, INC.",10.0,14.0,140.00,6,Velox,NanoSerum - 7.0mL,11,NanoSerum,0,0,0,7.12,Velox - NanoSerum,71.20,0.0,0.00,0.0,140.0,68.80,7.0
1836,Invoice,2021-06-25,E2110721,Velox - NanoSerum - 7.0mL,"LAHC, INC.",10.0,0.0,0.10,6,Velox,NanoSerum - 7.0mL,11,NanoSerum,1,1,0,7.12,Velox - NanoSerum,71.20,10.0,71.20,0.0,0.0,-71.20,0.0
1837,Invoice,2021-07-22,E2216843,Velox - NanoSerum - 7.0mL,Patients Corp,2.0,0.0,0.02,6,Velox,NanoSerum - 7.0mL,11,NanoSerum,1,1,0,7.12,Velox - NanoSerum,14.24,2.0,14.24,0.0,0.0,-14.24,0.0


In [ ]:
prodtypeDf = pd.DataFrame(dfMerge['Product type'].unique())
# List of All Unique Product Types
prodtypeDf.to_excel(r"UNIQUE Products List.xlsx",index=False,header=True)
prodtypeDf

,0
0,Flower Smalls
1,BHO
2,Diamonds
3,Disposable Cartridge
4,Dist Cart
5,Doser
6,Flower
7,Pre
8,Sauce Cart
9,Sugar


In [ ]:
# Is a specific Type
dfMerge['is Flower'] = np.where(dfMerge['Product type'].str.contains("Flower|flower|Weed|Shake"),1,0)
dfMerge['is Concentrate'] = np.where(dfMerge['Product type'].str.contains("BHO|Badder|Shatter|Diamonds|Sugar|Terp Glass|Distillate|Cartridge|Cart|Vape|Doser"),1,0)
dfMerge['is Pre Roll'] = np.where((dfMerge['Product type'].str.contains("Pre|Roll|Preroll|pk|6pk|1.0g")==True) & (dfMerge["Product type"].str.contains('Infused')==False),1,0)
dfMerge['Other'] = np.where(dfMerge['Product type'].str.contains("Evolve|3:1|Sativa|sativa|Hybrid|hybrid|Indica|indica|Kaviar|Infused|Nano|Moon|1.5g|NanoSerum"),1,0)

# Actual Number of Samples per Type 
dfMerge['Flower Sample'] = dfMerge['sample'] * dfMerge['Qty'] * dfMerge['is Flower']
dfMerge['Concentrate Sample'] = dfMerge['sample'] * dfMerge['Qty'] * dfMerge['is Concentrate']
dfMerge['Pre Roll Sample'] = dfMerge['sample'] * dfMerge['Qty'] * dfMerge['is Pre Roll']
dfMerge['Other Samples'] = dfMerge['sample'] * dfMerge['Qty'] * dfMerge['Other']

In [ ]:
sampleCountDf = dfMerge.groupby(['Name','Num','Date']).sum().reset_index()

In [ ]:
dfMerge['Date'].max()

Timestamp('2021-08-09 00:00:00')

In [ ]:
F# Getting Profit and Profit margin by Customer for last x number of days 

# Can Change to How ever many Days by changing Number below 
days = 30 
cutoff = dfMerge['Date'].max() - pd.Timedelta(days=days)
cutoff_Df = dfMerge[dfMerge['Date'] > cutoff]
xNumDaysDf = cutoff_Df.groupby('Name').sum().reset_index()
xNumDaysDf['Profit Margin'] = (xNumDaysDf['Profit'] /xNumDaysDf['Real Amount'])
xNumDaysDf['Profit Margin'] = np.where(xNumDaysDf['Profit Margin'] > -1000,xNumDaysDf['Profit Margin'],0)

xNumProfitDf= xNumDaysDf.sort_values(by=['Profit'], ascending=False)
xNumPmDf = xNumDaysDf.sort_values(by=['Profit Margin'], ascending=False)

In [ ]:
# Products without COGs
noCOGs = dfMerge.loc[dfMerge['COGs'] == 0]
listofNoCOGs = pd.DataFrame(noCOGs['Memo'].unique())
listofNoCOGs.to_excel(r"Prod_With_No_COGs.xlsx",index=False,header=True)
listofNoCOGs

,0
0,Dab Daddy - Flower - Orange Blast - 3.5g
1,Dab Daddy - Flower Smalls - Orange Blast - 7.0g
2,Dab Daddy - Pre-Roll - Granimals - 1.0g
3,Dab Daddy - Pre-Roll 3pk - Venom OG - 3 x 0.7g
4,Octavius - Flower - Granimals - 3.5g
5,Octavius - Flower - Strawberry Banana - 3.5g
6,Octavius - Flower - Wedding Cake - 3.5g


In [ ]:
# Set up Dates as Monthly
dfMerge.Date = pd.to_datetime(dfMerge.Date).dt.strftime('%Y/%m')

In [ ]:
# creating the overall company by Month 
overallDf = dfMerge.groupby('Date').sum().reset_index()
overallDf['Profit Margin'] = (overallDf['Profit'] / overallDf['Real Amount'])
startDate = '2021/01'
overallDf = overallDf.loc[overallDf['Date'] >= startDate]
overallDf

,Date,Qty,Sales Price,Amount,pos,pos P,sample,Small Sample,BOGO,COGs,Total COGs,Samples in Order,Small Sample COGs,BOGO COGs,Real Amount,Profit,Sales Commission,is Flower,is Concentrate,is Pre Roll,Other,Flower Sample,Concentrate Sample,Pre Roll Sample,Other Samples,Profit Margin
0,2021/01,2771.0,702.01746,35943.81984,1128,1181,83,81,2,923.52,19772.01,462.0,2444.39,602.00,35939.19984,16167.18984,1796.959992,49,61,6,14,53.0,284.0,114.0,11.0,0.449848
1,2021/02,9123.0,607.51700,133080.50200,1445,1417,114,112,2,1144.51,70000.73,679.0,4183.15,787.60,133073.71200,63072.98200,6653.685600,48,79,17,11,152.0,354.0,86.0,87.0,0.473970
2,2021/03,5318.0,914.30000,48707.95000,1707,1586,109,109,0,1323.77,40088.36,995.0,7374.77,0.00,48698.00000,8609.64000,2434.900000,43,116,12,4,138.0,678.0,169.0,10.0,0.176797
3,2021/04,8164.0,2241.50000,88287.21000,3672,3134,237,227,10,2941.34,60604.60,2341.0,10862.94,4266.70,88263.80000,27659.20000,4413.190000,124,191,51,17,103.0,1616.0,544.0,78.0,0.313370
4,2021/05,7674.0,1997.90000,74714.87000,3256,2871,212,202,10,2716.01,51804.08,2087.0,7017.16,3814.35,74694.00000,22889.92000,3734.700000,135,139,50,13,122.0,900.0,984.0,81.0,0.306449
5,2021/06,11744.0,7621.95000,114441.93500,2630,2343,128,122,6,2216.09,90125.26,766.0,3312.11,1447.00,114434.27500,24309.01500,5721.713750,84,148,29,2,63.0,440.0,253.0,10.0,0.212428
6,2021/07,6512.0,2134.25000,77553.73000,2613,2372,139,119,20,2144.13,48502.75,1923.0,3862.40,6159.00,77534.50000,29031.75000,3876.725000,100,113,47,3,118.0,960.0,843.0,2.0,0.374437
7,2021/08,3413.0,1469.25000,54330.53000,1332,1129,46,44,2,988.63,24527.98,253.0,512.24,1177.40,54328.00000,29800.02000,2716.400000,51,64,16,2,23.0,120.0,9.0,101.0,0.548520


In [ ]:
# get Orders by Customer 

In [ ]:
# Overall Orders by customer by date
CustOrders = dfMerge.groupby(['Name','Num','Date']).sum().reset_index()
CustOrders = CustOrders.drop(['Qty','Sales Price','pos','COGs'], axis=1)
CustOrders['Delivery Fee'] = 125
CustOrders['Profit'] = CustOrders['Profit'] - CustOrders['Delivery Fee'] - CustOrders['Sales Commission']
CustOrders['Profit Margin'] = (CustOrders['Profit'] /CustOrders['Real Amount']) 
CustOrders['Profit Margin'] = np.where(CustOrders['Profit Margin'] > -1000,CustOrders['Profit Margin'],0)


CustOrders

,Name,Num,Date,Amount,pos P,sample,Small Sample,BOGO,Total COGs,Samples in Order,Small Sample COGs,BOGO COGs,Real Amount,Profit,Sales Commission,is Flower,is Concentrate,is Pre Roll,Other,Flower Sample,Concentrate Sample,Pre Roll Sample,Other Samples,Delivery Fee,Profit Margin
0,420 Express Delivery Inc,E1881806,2021/05,3424.00,183,0,0,0,2009.28,0.0,0.00,0.0,3424.0,1118.52,171.2,2,20,0,0,0.0,0.0,0.0,0.0,125,0.326671
1,420 For the People Cooperative,E1420667,2021/01,1536.00,39,0,0,0,696.96,0.0,0.00,0.0,1536.0,637.24,76.8,3,0,0,0,0.0,0.0,0.0,0.0,125,0.414870
2,"Accucanna, LLC",E1958921,2021/05,0.22,161,18,18,0,190.96,22.0,190.96,0.0,0.0,-315.96,0.0,8,6,3,1,9.0,8.0,3.0,2.0,125,0.000000
3,Area 29 LLC,E1758095,2021/04,0.43,388,43,43,0,323.59,43.0,323.59,0.0,0.0,-448.59,0.0,14,19,6,4,14.0,19.0,6.0,4.0,125,0.000000
4,Area 29 LLC,E1877550,2021/04,1728.00,24,0,0,0,760.32,0.0,0.00,0.0,1728.0,756.28,86.4,3,0,0,0,0.0,0.0,0.0,0.0,125,0.437662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,Valley Herbal Center,E2018368,2021/06,1808.00,54,0,0,0,988.94,0.0,0.00,0.0,1808.0,603.66,90.4,1,6,0,0,0.0,0.0,0.0,0.0,125,0.333883
204,Valley Herbal Center,E2018620,2021/06,0.11,49,6,6,0,90.90,11.0,90.90,0.0,0.0,-215.90,0.0,1,5,0,0,2.0,9.0,0.0,0.0,125,0.000000
205,"Valley Herbal Healing Center, I",E2199343,2021/07,2144.25,106,5,5,0,1357.76,25.0,220.00,0.0,2144.0,554.04,107.2,6,0,4,0,15.0,0.0,10.0,0.0,125,0.258414
206,"Valley Herbal Healing Center, I",E2286015,2021/08,1744.00,28,0,0,0,225.20,0.0,0.00,0.0,1744.0,1306.60,87.2,3,0,1,0,0.0,0.0,0.0,0.0,125,0.749197


In [ ]:
# how many customers order x times - change this to include different people in Sheet
countOfOrders = CustOrders.groupby('Name').count()['Num'].reset_index()
moreThanXOrders = countOfOrders[countOfOrders['Num']>=6]
moreThanXOrders.count()['Name'] 

12

In [ ]:
# Creating OVerall By Customer 

someTig = CustOrders.groupby(['Name','Date']).sum().reset_index()
someTig['Shelving Fee'] = np.where(someTig.Name.str.contains('LAHC'),1500,0)
someTig["Profit"] = someTig['Profit'] - someTig['Shelving Fee'] 
someTig['Profit Margin'] = (someTig['Profit'] / someTig['Real Amount'])
someTig['Profit Margin'] = np.where(someTig['Profit Margin'] > -1000,someTig['Profit Margin'],0)
#make_float = lambda x: "{:,.2f}%".format(x)
#someTig['Profit Margin'] = someTig['Profit Margin'].apply(make_float)
someTig['Total Sample COGs'] = someTig['Small Sample COGs'] + someTig['BOGO COGs']

In [ ]:
overallRev = someTig['Real Amount'].sum()
print(overallRev)
overallProfit = someTig['Profit'].sum()
print(overallProfit)
overallPM = (overallProfit / overallRev) 
print(overallPM)


626965.48684
152191.44249800005
0.2427429351256123


In [ ]:
# Get Strictly numbers by customer 

In [ ]:
#Overall by Customer 
Customers = dfMerge.groupby('Name').sum().reset_index()
Customers = Customers.drop(['Qty','Sales Price','pos','COGs'], axis=1)
# individual customer check
gtg = Customers[Customers['Name']=='LAHC, INC.'].reset_index()
gtg
Customers

,Name,Amount,pos P,sample,Small Sample,BOGO,Total COGs,Samples in Order,Small Sample COGs,BOGO COGs,Real Amount,Profit,Sales Commission,is Flower,is Concentrate,is Pre Roll,Other,Flower Sample,Concentrate Sample,Pre Roll Sample,Other Samples
0,420 Express Delivery Inc,3424.00000,183,0,0,0,2009.28,0.0,0.00,0.00,3424.00000,1414.72000,171.200000,2,20,0,0,0.0,0.0,0.0,0.0
1,420 For the People Cooperative,1536.00000,39,0,0,0,696.96,0.0,0.00,0.00,1536.00000,839.04000,76.800000,3,0,0,0,0.0,0.0,0.0,0.0
2,"Accucanna, LLC",0.22000,161,18,18,0,190.96,22.0,190.96,0.00,0.00000,-190.96000,0.000000,8,6,3,1,9.0,8.0,3.0,2.0
3,Area 29 LLC,1728.43000,412,43,43,0,1083.91,43.0,323.59,0.00,1728.00000,644.09000,86.400000,17,19,6,4,14.0,19.0,6.0,4.0
4,Ash's First LLC,1728.00000,24,0,0,0,760.32,0.0,0.00,0.00,1728.00000,967.68000,86.400000,3,0,0,0,0.0,0.0,0.0,0.0
5,Atrium,3450.00000,110,0,0,0,1950.00,0.0,0.00,0.00,3450.00000,1500.00000,172.500000,0,5,0,0,0.0,0.0,0.0,0.0
6,Becc Holdings,91614.31200,348,22,22,0,43613.22,60.0,510.86,0.00,91613.71200,48000.49200,4580.685600,16,18,6,0,12.0,44.0,4.0,0.0
7,Cannaco Research Corporation,26859.37000,424,10,10,0,13884.75,137.0,1036.11,0.00,26858.00000,12973.25000,1342.900000,13,38,5,0,0.0,137.0,0.0,0.0
8,"Caps Apothecary, INC",0.16000,116,16,16,0,111.91,16.0,111.91,0.00,0.00000,-111.91000,0.000000,3,13,0,0,3.0,13.0,0.0,0.0
9,Connected Transportation Partne,2472.00000,20,0,0,0,1188.88,0.0,0.00,0.00,2472.00000,1283.12000,123.600000,1,0,3,0,0.0,0.0,0.0,0.0


In [ ]:
# Length
for i in Customers.Name:
  x = someTig[someTig['Name'] == i]
  chartLength = len(x['Amount'] >=3)
  print(chartLength)

1
1
1
1
1
1
4
2
1
1
3
1
1
2
2
1
1
1
2
2
1
4
1
2
1
1
2
8
1
1
1
5
1
1
1
1
1
1
6
1
1
6
1
1
2
5
2
4
1
5
3
1
1
1
1
4
2
4
2
1


In [ ]:
CustOrderWProd = dfMerge.groupby(['Name','Num','Date','Company','Product']).sum().reset_index()
CustOrderWProd['Profit Margin'] = (CustOrderWProd['Profit'] / CustOrderWProd['Real Amount'])
CustOrderWProd['Profit Margin'] = np.where(CustOrderWProd['Profit Margin'] > 0,CustOrderWProd['Profit Margin'],0)

CustOrderWProd[20:40]

,Name,Num,Date,Company,Product,Qty,Sales Price,Amount,pos,pos P,sample,Small Sample,BOGO,COGs,Total COGs,Samples in Order,Small Sample COGs,BOGO COGs,Real Amount,Profit,Sales Commission,is Flower,is Concentrate,is Pre Roll,Other,Flower Sample,Concentrate Sample,Pre Roll Sample,Other Samples,Profit Margin
20,420 Express Delivery Inc,E1881806,2021/05,Superbad,Flower - superB - 3.5g,32.0,16.0,512.00,9,8,0,0,0,8.02,256.64,0.0,0.00,0.0,512.0,255.36,25.6,1,0,0,0,0.0,0.0,0.0,0.0,0.49875
21,420 Express Delivery Inc,E1881806,2021/05,Superbad,Flower - superCRITICAL - 3.5g,32.0,16.0,512.00,9,8,0,0,0,8.02,256.64,0.0,0.00,0.0,512.0,255.36,25.6,1,0,0,0,0.0,0.0,0.0,0.0,0.49875
22,420 For the People Cooperative,E1420667,2021/01,Trees,Blunt Weed - Ali Bomaye - 3.5g,32.0,16.0,512.00,6,13,0,0,0,7.26,232.32,0.0,0.00,0.0,512.0,279.68,25.6,1,0,0,0,0.0,0.0,0.0,0.0,0.54625
23,420 For the People Cooperative,E1420667,2021/01,Trees,Blunt Weed - Blood Orange - 3.5g,32.0,16.0,512.00,6,13,0,0,0,7.26,232.32,0.0,0.00,0.0,512.0,279.68,25.6,1,0,0,0,0.0,0.0,0.0,0.0,0.54625
24,420 For the People Cooperative,E1420667,2021/01,Trees,Blunt Weed - Capped - 3.5g,32.0,16.0,512.00,6,13,0,0,0,7.26,232.32,0.0,0.00,0.0,512.0,279.68,25.6,1,0,0,0,0.0,0.0,0.0,0.0,0.54625
25,"Accucanna, LLC",E1958921,2021/05,Dab Daddy,BHO - Badder - Gasoline Alley - 1.0g,2.0,0.0,0.02,10,5,1,1,0,6.97,13.94,2.0,13.94,0.0,0.0,-13.94,0.0,0,1,0,0,0.0,2.0,0.0,0.0,0.00000
26,"Accucanna, LLC",E1958921,2021/05,Dab Daddy,Diamonds - Garanimals - 1.0g,1.0,0.0,0.01,10,10,1,1,0,8.00,8.00,1.0,8.00,0.0,0.0,-8.00,0.0,0,1,0,0,0.0,1.0,0.0,0.0,0.00000
27,"Accucanna, LLC",E1958921,2021/05,Dab Daddy,Diamonds - Orange Creamsicle - 1.0g,1.0,0.0,0.01,10,10,1,1,0,8.00,8.00,1.0,8.00,0.0,0.0,-8.00,0.0,0,1,0,0,0.0,1.0,0.0,0.0,0.00000
28,"Accucanna, LLC",E1958921,2021/05,Dab Daddy,Flower - Lava Cake - 3.5g,1.0,0.0,0.01,10,8,1,1,0,7.92,7.92,1.0,7.92,0.0,0.0,-7.92,0.0,1,0,0,0,1.0,0.0,0.0,0.0,0.00000
29,"Accucanna, LLC",E1958921,2021/05,Dab Daddy,Flower - Wedding Cake - 3.5g,1.0,0.0,0.01,10,8,1,1,0,7.92,7.92,1.0,7.92,0.0,0.0,-7.92,0.0,1,0,0,0,1.0,0.0,0.0,0.0,0.00000


In [ ]:
# Creating File 
workbook1 = xlsxwriter.Workbook('Vertical_Customers.xlsx')

# Formats 
format1 = workbook1.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})

# Add a format. Green fill with dark green text.
format2 = workbook1.add_format({'bg_color': '#C6EFCE',
                               'font_color': '#006100'})

bold = workbook1.add_format({'bold':1})
cell_format = workbook1.add_format()
cell_format.set_num_format(9)

currancy_format = workbook1.add_format()
currancy_format.set_num_format('$#,##0')


# overall worksheet 
overallWs = workbook1.add_worksheet('Overall')

overallLength = (len(overallDf['Date']) + 1)



overallData = [overallDf['Date'],overallDf['Profit'],overallDf['Profit Margin'],overallDf['Real Amount'],overallDf['Total COGs'],overallDf['Small Sample'], overallDf['Small Sample COGs'],overallDf['BOGO'], overallDf['BOGO COGs'],overallDf['Sales Commission'],overallDf['Flower Sample'],overallDf['Concentrate Sample'], overallDf['Pre Roll Sample'], overallDf['Other Samples']]
overallHeadings = ['Date','Profit','Profit Margin', 'Real Amount', 'Total COGs', 'Small Sample', 'Small Sample COGs', 'BOGO', 'BOGO COGs', 'Sales Commission', 'Flower Sample', 'Concentrate Sample', 'Pre Roll Sample','Other Samples']
overallWs.write_row('A1', overallHeadings, bold)
overallWs.write_column('A2', overallData[0])
overallWs.write_column('B2', overallData[1],currancy_format)
overallWs.write_column('C2', overallData[2], cell_format)
overallWs.write_column('D2', overallData[3], currancy_format)
overallWs.write_column('E2', overallData[4], currancy_format)
overallWs.write_column('F2', overallData[5])
overallWs.write_column('G2', overallData[6], currancy_format)
overallWs.write_column('H2', overallData[7])
overallWs.write_column('I2', overallData[8], currancy_format)
overallWs.write_column('J2', overallData[9], currancy_format)
overallWs.write_column('K2', overallData[10])
overallWs.write_column('L2', overallData[11])
overallWs.write_column('M2', overallData[12])
overallWs.write_column('N2', overallData[13])

ocell_formatB = 'B2:B'+str(overallLength)
ocell_formatC = 'C2:C'+str(overallLength)
ocell_formatD = 'D2:D'+str(overallLength)
ocell_formatE = 'E2:E'+str(overallLength)
ocell_formatJ = 'J2:J'+str(overallLength)
ocell_formatG = 'G2:G'+str(overallLength)
ocell_formatH = 'H2:H'+str(overallLength)
ocell_formatI = 'I2:I'+str(overallLength)
ocell_formatK = 'K2:K'+str(overallLength)
ocell_formatL = 'L2:L'+str(overallLength)
ocell_formatM = 'M2:M'+str(overallLength)
ocell_formatF = 'F2:F'+str(overallLength)
ocell_formatN = 'N2:N'+str(overallLength)

overallWs.write_formula('B'+(str(overallLength+2)), '=SUM('+(str(ocell_formatB))+')',currancy_format)
overallWs.write_formula('F'+(str(overallLength+2)), '=SUM('+(str(ocell_formatF))+')')
overallWs.write_formula('D'+(str(overallLength+2)), '=SUM('+(str(ocell_formatD))+')',currancy_format)
overallWs.write_formula('E'+(str(overallLength+2)), '=SUM('+(str(ocell_formatE))+')',currancy_format)
overallWs.write_formula('G'+(str(overallLength+2)), '=SUM('+(str(ocell_formatG))+')',currancy_format)
overallWs.write_formula('H'+(str(overallLength+2)), '=SUM('+(str(ocell_formatH))+')')
overallWs.write_formula('I'+(str(overallLength+2)), '=SUM('+(str(ocell_formatI))+')',currancy_format)
overallWs.write_formula('J'+(str(overallLength+2)), '=SUM('+(str(ocell_formatJ))+')',currancy_format)
overallWs.write_formula('K'+(str(overallLength+2)), '=SUM('+(str(ocell_formatK))+')')
overallWs.write_formula('L'+(str(overallLength+2)), '=SUM('+(str(ocell_formatL))+')')
overallWs.write_formula('M'+(str(overallLength+2)), '=SUM('+(str(ocell_formatM))+')')
overallWs.write_formula('N'+(str(overallLength+2)), '=SUM('+(str(ocell_formatN))+')')
overallWs.write_formula('C'+(str(overallLength+2)), '=B'+(str(overallLength+2))+'/'+'D'+(str(overallLength+2)), cell_format)
overallWs.write('A'+(str(overallLength+2)), 'Total:', bold)
overallWs.set_column('A:M',11)




# Add first worksheet 
firstWorkSheet = workbook1.add_worksheet('Customers profit')
profitSheetDataP = [xNumProfitDf['Name'],xNumProfitDf['Profit'],xNumProfitDf['Real Amount'],xNumProfitDf['Total COGs'],xNumProfitDf['Profit Margin']]
profitSheetDataPM = [xNumPmDf['Name'],xNumPmDf['Profit Margin'],xNumPmDf['Real Amount'],xNumPmDf['Total COGs'],xNumPmDf['Profit']]
profitHeadings = ['Company','Profit Margin','Profit','Revenue','COGs (Internal)','','Company','Profit','Profit Margin','Revenue','COGs (Internal)']
firstWorkSheet.write_row('A1',profitHeadings,bold)
firstWorkSheet.write_column('A2',profitSheetDataPM[0])
firstWorkSheet.write_column('B2',profitSheetDataPM[1], cell_format)
firstWorkSheet.write_column('C2',profitSheetDataPM[4], currancy_format)
firstWorkSheet.write_column('D2',profitSheetDataPM[2], currancy_format)
firstWorkSheet.write_column('E2',profitSheetDataPM[3], currancy_format)
#firstWorkSheet.write_column('E2',custData[4])
firstWorkSheet.write_column('G2',profitSheetDataP[0])
firstWorkSheet.write_column('H2',profitSheetDataP[1],currancy_format)
firstWorkSheet.write_column('I2',profitSheetDataP[4],cell_format)
firstWorkSheet.write_column('J2',profitSheetDataPM[2], currancy_format)
firstWorkSheet.write_column('K2',profitSheetDataPM[3], currancy_format)


lenG = len(xNumPmDf['Profit Margin'])
firstWorkSheet.conditional_format('B2:B'+str(lenG+1), {'type':     'cell',
                                        'criteria': '>=',
                                        'value':    overallPM,
                                        'format':   format2})

firstWorkSheet.conditional_format('B2:B'+str(lenG+1), {'type':     'cell',
                                        'criteria': '<',
                                        'value':    overallPM,
                                        'format':   format1})
firstWorkSheet.conditional_format('I2:I'+str(lenG+1), {'type':     'cell',
                                        'criteria': '>=',
                                        'value':    overallPM,
                                        'format':   format2})

firstWorkSheet.conditional_format('I2:I'+str(lenG+1), {'type':     'cell',
                                        'criteria': '<',
                                        'value':    overallPM,
                                        'format':   format1})

firstWorkSheet.set_column('A:L',11)

# add charts for first worksheet 

# Creating a sheet with each Company Name on it
for i in moreThanXOrders.Name:
  worksheet = workbook1.add_worksheet(i)
  # bold format 
  bold = workbook1.add_format({'bold':1})
  
  
  # Headings
  headings = ['Date','Revenue','Total COGs','Sales Com','Profit','Profit Margin','Samples in Order','Small Sample','Small Sample COGS','BOGO','BOGO COGs','Total Sample COGs','Flower Sample','Concentrate Sample','Pre Roll Sample','Other Sample','Delivery Fees','Shelving Fee'] 
  headings2 = ['Num','Date','Company','Product','Qty','Sales Price','Revenue','Sample','COGs','Total COGs','Samples in Order','Profit','Profit Margin','Sales Commission']
  headings3 = ['Num','Date','Revenue','Sample','Total COGs','Samples in Order','Profit','Profit Margin','Sales Commission','Delivery Fee']
  
  # Data 1
  x = someTig[someTig['Name'] == i]
  xx = sampleCountDf[sampleCountDf['Name']== i]
  # Length Automation stuff
  chartLength = len(x['Date'])
  if chartLength >= 14:
    data2start = chartLength + 5
  else:
    data2start = 17
  head2Start = data2start - 1

  #data 1 
  
  data= [x['Date'],x['Real Amount'],x['Total COGs'],x['Sales Commission'],x['Profit'],x['Profit Margin'],x['Samples in Order'],x['Small Sample'],x['Small Sample COGs'],x['BOGO'],x['BOGO COGs'],x['Total Sample COGs'],x['Flower Sample'],x['Concentrate Sample'],x['Pre Roll Sample'],x['Other Samples'],x['Delivery Fee'],x['Shelving Fee']]
  # Data 2
  v = CustOrderWProd[CustOrderWProd['Name'] == i]
  mask = v['Num'].ne(v['Num'].shift(-1))
  s = pd.DataFrame('',index=mask.index[mask] + .5, columns=v.columns)
  rt = pd.concat([v, s]).sort_index().reset_index(drop=True).iloc[:-1]
  data2 = [rt['Num'],rt['Date'],rt['Company'],rt['Product'],rt['Qty'],rt['Sales Price'],rt['Real Amount'],rt['sample'],rt['COGs'],rt['Total COGs'],rt['Samples in Order'],rt['Profit'],rt['Profit Margin'],rt['Sales Commission']]

  fgh = rt[rt['Num'] == ''].index
  for index in fgh:
    headingIndex = index + data2start
    worksheet.write_row('A'+str(headingIndex),headings2,bold)

  # Data 3
  h = CustOrders[CustOrders['Name']==i]
  data3 = [h['Num'],h['Date'],h['Real Amount'],h['sample'],h['Total COGs'],h['Samples in Order'],h['Profit'], h['Profit Margin'], h['Sales Commission'],h['Delivery Fee']]

  


  # ADDING SUM TO ALL MONTH STUFF
  cell_formatB = 'B2:B'+str(chartLength+1)
  cell_formatC = 'C2:C'+str(chartLength+1)
  cell_formatD = 'D2:D'+str(chartLength+1)
  cell_formatE = 'E2:E'+str(chartLength+1)
  cell_formatJ = 'J2:J'+str(chartLength+1)
  cell_formatG = 'G2:G'+str(chartLength+1)
  cell_formatH = 'H2:H'+str(chartLength+1)
  cell_formatI = 'I2:I'+str(chartLength+1)
  cell_formatK = 'K2:K'+str(chartLength+1)
  cell_formatL = 'L2:L'+str(chartLength+1)
  cell_formatM = 'M2:M'+str(chartLength+1)
  cell_formatN = 'N2:N'+str(chartLength+1)
  cell_formatO = 'O2:O'+str(chartLength+1)
  cell_formatP = 'P2:P'+str(chartLength+1)
  cell_formatQ = 'Q2:Q'+str(chartLength+1)
  cell_formatR = 'R2:R'+str(chartLength+1)

  # Get totals for columns 
  worksheet.write_formula('B'+(str(chartLength+2)), '=SUM('+(str(cell_formatB))+')',currancy_format)
  worksheet.write_formula('C'+(str(chartLength+2)), '=SUM('+(str(cell_formatC))+')',currancy_format)
  worksheet.write_formula('D'+(str(chartLength+2)), '=SUM('+(str(cell_formatD))+')',currancy_format)
  worksheet.write_formula('E'+(str(chartLength+2)), '=SUM('+(str(cell_formatE))+')',currancy_format)
  worksheet.write_formula('G'+(str(chartLength+2)), '=SUM('+(str(cell_formatG))+')')
  worksheet.write_formula('H'+(str(chartLength+2)), '=SUM('+(str(cell_formatH))+')')
  worksheet.write_formula('I'+(str(chartLength+2)), '=SUM('+(str(cell_formatI))+')',currancy_format)
  worksheet.write_formula('J'+(str(chartLength+2)), '=SUM('+(str(cell_formatJ))+')')
  worksheet.write_formula('K'+(str(chartLength+2)), '=SUM('+(str(cell_formatK))+')',currancy_format)
  worksheet.write_formula('L'+(str(chartLength+2)), '=SUM('+(str(cell_formatL))+')',currancy_format)
  worksheet.write_formula('M'+(str(chartLength+2)), '=SUM('+(str(cell_formatM))+')')
  worksheet.write_formula('N'+(str(chartLength+2)), '=SUM('+(str(cell_formatN))+')')
  worksheet.write_formula('O'+(str(chartLength+2)), '=SUM('+(str(cell_formatO))+')')
  worksheet.write_formula('P'+(str(chartLength+2)), '=SUM('+(str(cell_formatP))+')')
  worksheet.write_formula('Q'+(str(chartLength+2)), '=SUM('+(str(cell_formatQ))+')')
  worksheet.write_formula('R'+(str(chartLength+2)), '=SUM('+(str(cell_formatR))+')')
  worksheet.write_formula('F'+(str(chartLength+2)), '=E'+(str(chartLength+2))+'/'+'B'+(str(chartLength+2)), cell_format)
  # doesn't average bc of % sign 
  #worksheet.write_formula('F'+(str(chartLength+2)), '=AVERAGE('+(str(cell_formatF))+')')
  worksheet.write('A'+(str(chartLength+2)),'Total:',bold)
  # Data 1 conditional 
  worksheet.conditional_format('F2:F'+(str(chartLength+2)), {'type':     'cell',
                                        'criteria': '>=',
                                        'value':    overallPM,
                                        'format':   format2})
  worksheet.conditional_format('F2:F'+(str(chartLength+2)), {'type':     'cell',
                                        'criteria': '<',
                                        'value':    overallPM,
                                        'format':   format1})

  # Create headings 
  worksheet.write_row('A1',headings,bold)
  worksheet.write_row('A'+str(head2Start),headings2,bold)
  worksheet.write_row('Q'+str(head2Start),headings3,bold)

# Write a column starting at point specified 
# Overall Orders - Data 1 
  worksheet.write_column('A2',data[0])
  worksheet.write_column('B2',data[1], currancy_format)
  worksheet.write_column('C2',data[2], currancy_format)
  worksheet.write_column('D2',data[3], currancy_format)
  worksheet.write_column('E2',data[4], currancy_format)
  worksheet.write_column('F2',data[5],cell_format)
  worksheet.write_column('G2',data[6])
  worksheet.write_column('H2',data[7])
  worksheet.write_column('I2',data[8], currancy_format)
  worksheet.write_column('J2',data[9])
  worksheet.write_column('K2',data[10], currancy_format)
  worksheet.write_column('L2',data[11], currancy_format)
  worksheet.write_column('M2',data[12])
  worksheet.write_column('N2',data[13])
  worksheet.write_column('O2',data[14])
  worksheet.write_column('P2',data[15])
  worksheet.write_column('Q2',data[16])
  worksheet.write_column('R2',data[17])
  # Data 2  
  worksheet.write_column('A'+str(data2start),data2[0])
  worksheet.write_column('B'+str(data2start),data2[1])
  worksheet.write_column('C'+str(data2start),data2[2])
  worksheet.write_column('D'+str(data2start),data2[3])
  worksheet.write_column('E'+str(data2start),data2[4])
  worksheet.write_column('F'+str(data2start),data2[5])
  worksheet.write_column('G'+str(data2start),data2[6])
  worksheet.write_column('H'+str(data2start),data2[7])
  worksheet.write_column('I'+str(data2start),data2[8])
  worksheet.write_column('J'+str(data2start),data2[9])
  worksheet.write_column('K'+str(data2start),data2[10])
  worksheet.write_column('L'+str(data2start),data2[11])
  worksheet.write_column('M'+str(data2start),data2[12],cell_format)
  worksheet.write_column('N'+str(data2start),data2[13])
  # Data 3 
  worksheet.write_column('Q'+str(data2start),data3[0])
  worksheet.write_column('R'+str(data2start),data3[1])
  worksheet.write_column('S'+str(data2start),data3[2])
  worksheet.write_column('T'+str(data2start),data3[3])
  worksheet.write_column('U'+str(data2start),data3[4])
  worksheet.write_column('V'+str(data2start),data3[5])
  worksheet.write_column('W'+str(data2start),data3[6])
  worksheet.write_column('X'+str(data2start),data3[7],cell_format)
  worksheet.write_column('Y'+str(data2start),data3[8])
  worksheet.write_column('Z'+str(data2start),data3[9])

  lenData3 = len(h['Profit Margin']) + data2start - 1
  worksheet.conditional_format('X'+(str(data2start))+':X'+str(lenData3), {'type':     'cell',
                                        'criteria': '>=',
                                        'value':    overallPM,
                                        'format':   format2})
  worksheet.conditional_format('X'+(str(data2start))+':X'+str(lenData3), {'type':     'cell',
                                        'criteria': '<',
                                        'value':    overallPM,
                                        'format':   format1})

  chart1 = workbook1.add_chart({'type':'column'})
  # Configure the first series.
  # = Sheet1 !$A$1 is equivalent to ['Sheet1', 0, 0].
  chart1.add_series({
      'name': [i,0,1],
      'categories': [i,1,0,chartLength,0],
      'values': [i,1,1,chartLength,1]
  })
  # Configure a second series.
  # Note use of alternative syntax to define ranges.
  # [sheetname, first_row, first_col, last_row, last_col].
  chart1.add_series({
      'name': [i,0,2],
      'categories': [i,1,0,chartLength,0],
      'values': [i,1,2,chartLength,2],
  })
  # Add a chart title 
  chart1.set_title({'name': 'Rev vs COGs by Month'})
  chart1.set_y_axis({'name':'$ Value'})
  # set y axis 
  chart1.set_x_axis({'name':'Month'})

  # Set excel chart style 
  chart1.set_style(11)
  # add chart to wroksheet 
  worksheet.insert_chart('AC1', chart1)

  chart2 = workbook1.add_chart({'type':'line'})
  chart2.add_series({
      'name': [i,0,4],
      'categories': [i,1,0,chartLength,0],
      'values': [i,1,4,chartLength,4],
  })
  chart2.set_title({'name':'Profit by Month'})
  chart2.set_y_axis({'name':'Profit'})
  # Set y
  chart2.set_x_axis({'name':'Date'})
  chart2.set_style(11)
  worksheet.insert_chart('T1',chart2)

  chart3 = workbook1.add_chart({'type':'line'})
  chart3.add_series({
      'name': [i,0,3],
      'categories': [i,1,0,chartLength,0],
      'values': [i,1,3,chartLength,3]
  })
  chart3.set_title({'name':'Samples by Month'})
  chart3.set_x_axis({'name':'Date'})
  chart3.set_y_axis({'name':'Samples'})
  chart3.set_style(11)
  worksheet.insert_chart('AJ1',chart3)
  
  worksheet.set_column('A:R',11)

  
workbook1.close()
